## 84. Largest Rectangle in Histogram
- Description:
  <blockquote>
  Given an array of integers `heights` representing the histogram's bar height where the width of each bar is `1`, return _the area of the largest rectangle in the histogram_.

  **Example 1:**

  ![](https://assets.leetcode.com/uploads/2021/01/04/histogram.jpg)

  ```
  Input: heights = [2,1,5,6,2,3]
  Output: 10
  Explanation: The above is a histogram where width of each bar is 1.
  The largest rectangle is shown in the red area, which has an area = 10 units.

  ```

  **Example 2:**

  ![](https://assets.leetcode.com/uploads/2021/01/04/histogram-1.jpg)

  ```
  Input: heights = [2,4]
  Output: 4

  ```

  **Constraints:**

  -   `1 <= heights.length <= 10<sup>5</sup>`
  -   `0 <= heights[i] <= 10<sup>4</sup>`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/largest-rectangle-in-histogram)

- Topics: Stack

- Difficulty: Hard

- Resources: example_resource_URL

### Solution 1, Optimised brute force with nested for loops
We use two nested loops to iterate from current element to the end of the array to find the rectangle with the max possible area

- Time Complexity: O(N^2)
- Space Complexity: O(1)

In [ ]:
from math import inf
from typing import List


class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        max_area = 0
        for i in range(len(heights)):
            min_height = inf
            for j in range(i, len(heights)):
                min_height = min(min_height, heights[j])
                max_area = max(max_area, min_height * (j - i + 1))
        return max_area

### Solution 2, Divide and Conquer Approach
This approach relies on the observation that the rectangle with maximum area will be the maximum of:
1. The widest possible rectangle with height equal to the height of the shortest bar.

2. The largest rectangle confined to the left of the shortest bar(subproblem).

3. The largest rectangle confined to the right of the shortest bar(subproblem).

- Time Complexity: O(N log N)
  - Worst Case: O(n2). If the numbers in the array are sorted, we don't gain the advantage of divide and conquer.
- Space Complexity: O(N), Recursion with worst case depth n.

In [ ]:
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        def calculateArea(heights: List[int], start: int, end: int) -> int:
            if start > end:
                return 0
            min_index = start
            for i in range(start, end + 1):
                if heights[min_index] > heights[i]:
                    min_index = i
            return max(
                heights[min_index] * (end - start + 1),
                calculateArea(heights, start, min_index - 1),
                calculateArea(heights, min_index + 1, end),
            )

        return calculateArea(heights, 0, len(heights) - 1)

### Solution 3, Optimum Monotonically Increasing bar's index Stack Approach

For each bar, we want to answer: "What's the largest rectangle I can make using this bar's height?"
To answer this, we need to know:
- How far left can this bar extend?
- How far right can this bar extend?

A bar can extend in a direction as long as neighboring bars are at least as tall.

The Core Insight:  
When we encounter a shorter bar, we suddenly know the exact boundaries for all taller bars behind it!


The stack stores bars in increasing height order. This tells us:  

- When we pop a bar, the bar below it in the stack is the first shorter bar to its left
- The current position is the first shorter bar to its right
- So the width is: right_boundary - left_boundary - 1


Why This Works  
Instead of calculating rectangles bar-by-bar going forward, we calculate them when we discover their RIGHT boundary going forward.

We don't know how far right a bar extends until we hit something shorter
When we do, we immediately calculate and "close out" all affected bars
The stack gives us the LEFT boundary (previous smaller element)

Complexities:
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
from typing import List

class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        stack = [-1]
        max_area = 0
        
        for i in range(len(heights)):
            while stack[-1] != -1 and heights[stack[-1]] >= heights[i]:
                current_height = heights[stack.pop()]
                current_width = i - stack[-1] - 1
                max_area = max(max_area, current_height * current_width)
            
            stack.append(i)

        # Remaining indices in the stack represent bars that never encountered a shorter bar to their right.
        # In other words, they can extend all the way to the end of the array, or the beginning and the end
        while stack[-1] != -1:
            current_height = heights[stack.pop()]
            current_width = len(heights) - stack[-1] - 1
            max_area = max(max_area, current_height * current_width)
            
        return max_area
    
    
heights = [2, 1, 5, 6, 2, 3]
sol = Solution()
result = sol.largestRectangleArea(heights)
print(result)